In [ ]:
import pygame
import random
import os

pygame.init()

#  Основные настройки 
WIDTH, HEIGHT = 600, 400
BLOCK_SIZE = 20
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
RED = (200, 0, 0)
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
GRAY = (30, 30, 30)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")
clock = pygame.time.Clock()
font = pygame.font.SysFont("Arial", 24)
big_font = pygame.font.SysFont("Arial", 36)

# Путь и словарь результатов 
SAVE_FILE = "user_scores.txt"
user_scores = {}

if os.path.exists(SAVE_FILE):
    with open(SAVE_FILE, 'r') as file:
        for line in file:
            name, score = line.strip().split(',')
            user_scores[name] = int(score)

# Ввод имени и показ лучших результатов 
def enter_username_screen():
    input_box = pygame.Rect(WIDTH // 2 - 100, HEIGHT // 2, 200, 40)
    color_inactive = pygame.Color('lightskyblue3')
    color_active = pygame.Color('dodgerblue2')
    color = color_inactive
    active = False
    text = ''
    done = False
    global user_scores

    while not done:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return None
            if event.type == pygame.MOUSEBUTTONDOWN:
                if input_box.collidepoint(event.pos):
                    active = True
                    color = color_active
                else:
                    active = False
                    color = color_inactive
            if event.type == pygame.KEYDOWN:
                if active:
                    if event.key == pygame.K_RETURN and text:
                        return text
                    elif event.key == pygame.K_BACKSPACE:
                        text = text[:-1]
                    elif len(text) < 15:
                        text += event.unicode

        screen.fill(GRAY)

        title_msg = big_font.render("Enter your username:", True, WHITE)
        screen.blit(title_msg, (WIDTH // 2 - title_msg.get_width() // 2, HEIGHT // 2 - 80))

        txt_surface = font.render(text, True, color)
        screen.blit(txt_surface, (input_box.x + 5, input_box.y + 5))
        pygame.draw.rect(screen, color, input_box, 2)

        # Показ лучших результатов (последние 3 пользователя)
        y_offset = 10
        screen.blit(font.render("Previous Players:", True, WHITE), (10, y_offset))
        y_offset += 25
        for name, score in list(user_scores.items())[-3:]:
            msg = f"{name}: {score}"
            screen.blit(font.render(msg, True, WHITE), (10, y_offset))
            y_offset += 20

        pygame.display.flip()
        clock.tick(30)

#  Генерация стен 
def generate_walls():
    walls = []
    for x in range(0, WIDTH, BLOCK_SIZE):
        walls.append((x, 0))
        walls.append((x, HEIGHT - BLOCK_SIZE))
    for y in range(0, HEIGHT, BLOCK_SIZE):
        walls.append((0, y))
        walls.append((WIDTH - BLOCK_SIZE, y))
    return walls

#  Генерация еды 
def generate_food(snake, walls):
    while True:
        x = random.randint(1, (WIDTH // BLOCK_SIZE) - 2) * BLOCK_SIZE
        y = random.randint(1, (HEIGHT // BLOCK_SIZE) - 2) * BLOCK_SIZE
        pos = (x, y)
        if pos not in snake and pos not in walls:
            weight = random.choice([1, 2, 3])
            color = RED if weight == 1 else YELLOW
            timestamp = pygame.time.get_ticks()
            return {'pos': pos, 'weight': weight, 'color': color, 'time': timestamp}

# Основная игра 
def run_game(username):
    snake = [(100, 100)]
    direction = (BLOCK_SIZE, 0)
    walls = generate_walls()
    food = generate_food(snake, walls)
    score = 0
    level = 1
    speed = 10
    paused = False
    game_over = False
    FOOD_LIFETIME = 5000

    while True:
        if not paused:
            clock.tick(speed)
        else:
            clock.tick(5)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            if event.type == pygame.MOUSEBUTTONDOWN and game_over:
                if event.button == 1:
                    return True
            if event.type == pygame.KEYDOWN:
                if not game_over:
                    if event.key == pygame.K_UP and direction != (0, BLOCK_SIZE):
                        direction = (0, -BLOCK_SIZE)
                    elif event.key == pygame.K_DOWN and direction != (0, -BLOCK_SIZE):
                        direction = (0, BLOCK_SIZE)
                    elif event.key == pygame.K_LEFT and direction != (BLOCK_SIZE, 0):
                        direction = (-BLOCK_SIZE, 0)
                    elif event.key == pygame.K_RIGHT and direction != (-BLOCK_SIZE, 0):
                        direction = (BLOCK_SIZE, 0)
                    elif event.key == pygame.K_p:
                        paused = not paused
                        if paused:
                            save_score(username, score)
        if not game_over and not paused:
            head = (snake[0][0] + direction[0], snake[0][1] + direction[1])
            snake.insert(0, head)

            if (
                head in snake[1:] or
                head in walls or
                head[0] < 0 or head[0] >= WIDTH or
                head[1] < 0 or head[1] >= HEIGHT
            ):
                game_over = True
                save_score(username, score)
            elif head == food['pos']:
                score += food['weight']
                if score % 4 == 0:
                    level += 1
                    speed += 2
                food = generate_food(snake, walls)
            else:
                snake.pop()

            if pygame.time.get_ticks() - food['time'] > FOOD_LIFETIME:
                food = generate_food(snake, walls)

        screen.fill(BLACK)
        for wall in walls:
            pygame.draw.rect(screen, BLUE, (*wall, BLOCK_SIZE, BLOCK_SIZE))
        for part in snake:
            pygame.draw.rect(screen, GREEN, (*part, BLOCK_SIZE, BLOCK_SIZE))
        pygame.draw.rect(screen, food['color'], (*food['pos'], BLOCK_SIZE, BLOCK_SIZE))

        score_text = font.render(f"Score: {score}  Level: {level}", True, WHITE)
        screen.blit(score_text, (10, 10))

        if paused:
            pause_msg = big_font.render("Paused", True, WHITE)
            screen.blit(pause_msg, (WIDTH // 2 - pause_msg.get_width() // 2, HEIGHT // 2 - 40))

        if game_over:
            msg = big_font.render("Game Over!", True, WHITE)
            restart_msg = font.render("Click to Restart or Close", True, WHITE)
            screen.blit(msg, (WIDTH // 2 - msg.get_width() // 2, HEIGHT // 2 - 40))
            screen.blit(restart_msg, (WIDTH // 2 - restart_msg.get_width() // 2, HEIGHT // 2 + 10))

        pygame.display.flip()

# Сохранение очков
def save_score(username, score):
    global user_scores
    user_scores[username] = max(user_scores.get(username, 0), score)

    # Удалить самых старых пользователей, если их больше 3
    if len(user_scores) > 3:
        # Сортировка по времени добавления: сохранить последние 3
        user_scores = dict(list(user_scores.items())[-3:])
    
    with open(SAVE_FILE, 'w') as file:
        for name, score in user_scores.items():
            file.write(f"{name},{score}\n")

# Главный запуск 
username = enter_username_screen()
if username:
    while True:
        restart = run_game(username)
        if not restart:
            break

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
